In [ ]:
from torch import nn

class SVD(nn.Module):
    def __init__(self, num_factors, num_users, num_items, device, **kwargs):
        super(SVD, self).__init__(**kwargs)
        self.device = device
        # plain MF params
        self.P = nn.Embedding(num_users, num_factors).to(self.device)
        self.Q = nn.Embedding(num_items, num_factors).to(self.device)

    def forward(self, user_id, item_id):

        P_u = self.P(user_id)
        Q_i = self.Q(item_id)

        outputs = (P_u * Q_i).sum(axis=1)

        return outputs.flatten()


In [ ]:
num_factors = 5
batch_size = 128

l2_reg = 1e-5
lr = 0.001
num_epochs = 100

In [ ]:
model = SVD(num_factors, num_users, num_items, device)

loss_fn = nn.MSELoss(reduction='mean')

optimizer = torch.optim.Adam((param for param in model.parameters()
                              if param.requires_grad),
                              weight_decay=l2_reg,
                              lr=lr)

In [ ]:
for epoch in tqdm(range(num_epochs)):
    tr_rmse = 0

    model.train()
    for u, i, r in train_loader:

        u, i, r = u.to(device), i.to(device), r.to(device)

        optimizer.zero_grad()

        output = model(u, i)

        l = loss_fn(output, r)
        l.backward()
        optimizer.step()

        with torch.no_grad():
            tr_rmse += np.sqrt(loss_fn(output, r).cpu().numpy())